In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [1]:
import lightgbm as lgb

In [3]:
df_train = pd.read_csv('train.csv')

In [14]:
df_test = pd.read_csv('test.csv')
df_test = df_test.drop('ID_code',axis=1)

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
X = df_train.drop(['target','ID_code'],axis=1)
y = df_train['target']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33, random_state=42)

In [7]:
from sklearn.model_selection import StratifiedKFold

In [9]:
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits,shuffle=True,random_state=101)

In [16]:
count = 1

params = {'objective':'binary', 'metric':'auc', 'learning_rate': 0.01, 'num_rounds':6000, 'verbose':1}
test_pred = np.zeros(len(df_test))

for train_index,val_index in skf.split(X_train, y_train):
    print('Fold: {}'.format(count))
    lgb_train = lgb.Dataset(data = X_train.iloc[train_index], label = y_train.iloc[train_index])
    lgb_val = lgb.Dataset(data = X_train.iloc[val_index], label = y_train.iloc[val_index])
    
    model = lgb.train(params, train_set = lgb_train, valid_sets = lgb_val,verbose_eval=500)
    
    test_pred += model.predict(df_test)/n_splits
    
    count = count+1

Fold: 1


c:\users\madhusudan rudresh\appdata\local\programs\python\python36\lib\site-packages\lightgbm\engine.py:118: UserWarning: Found `num_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[500]	valid_0's auc: 0.849379
[1000]	valid_0's auc: 0.870792
[1500]	valid_0's auc: 0.880411
[2000]	valid_0's auc: 0.884718
[2500]	valid_0's auc: 0.887124
[3000]	valid_0's auc: 0.888704
[3500]	valid_0's auc: 0.889071
[4000]	valid_0's auc: 0.889188
[4500]	valid_0's auc: 0.889224
[5000]	valid_0's auc: 0.889245
[5500]	valid_0's auc: 0.889343
[6000]	valid_0's auc: 0.889309
Fold: 2
[500]	valid_0's auc: 0.845566
[1000]	valid_0's auc: 0.86829
[1500]	valid_0's auc: 0.877084
[2000]	valid_0's auc: 0.881823
[2500]	valid_0's auc: 0.884715
[3000]	valid_0's auc: 0.886338
[3500]	valid_0's auc: 0.886878
[4000]	valid_0's auc: 0.886883
[4500]	valid_0's auc: 0.886763
[5000]	valid_0's auc: 0.886616
[5500]	valid_0's auc: 0.886553
[6000]	valid_0's auc: 0.886364
Fold: 3
[500]	valid_0's auc: 0.852076
[1000]	valid_0's auc: 0.874377
[1500]	valid_0's auc: 0.883654
[2000]	valid_0's auc: 0.888332
[2500]	valid_0's auc: 0.890679
[3000]	valid_0's auc: 0.891765
[3500]	valid_0's auc: 0.892365
[4000]	valid_0's auc: 0.892

In [21]:
ID = pd.read_csv('test.csv')
ID = ID['ID_code']

In [29]:
lgbm_output = pd.DataFrame(data=test_pred,index=ID)
lgbm_output.columns = ['target']

In [30]:
lgbm_output.to_csv('output_5thmar.csv')